In [9]:
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install statsmodels
!{sys.executable} -m pip install googlemaps

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import statsmodels.api as sm

In [36]:
flights = pd.read_csv('./data/flights/all_flights.csv')
flights = flights.drop(columns=['FL_NUM', 'ORIGIN_CITY_MARKET_ID', 'DEST_CITY_MARKET_ID'])
flights = flights.loc[:, ~flights.columns.str.contains('^Unnamed')]

ufos = pd.read_csv("data/ufo-sightings/ufos-2013.csv", sep=',', low_memory=False)
ufos['lat'] = ufos['latitude']
ufos['lng'] = ufos['longitude']
ufos = ufos[[ 'datetime', 'duration (seconds)', 'lat', 'lng' ]]
ufos = ufos[ ufos['datetime'] > '2013-01-01 00:00:00' ]

In [39]:
ufos.head()

,datetime,duration (seconds),lat,lng
5,2013-01-01 00:01:00,180.0,45.523611,-122.675000
6,2013-01-01 00:02:00,120.0,30.411111,-88.827778
7,2013-01-01 00:02:00,60.0,18.473833,-66.938512
8,2013-01-01 00:04:00,300.0,-27.522349,153.191323
9,2013-01-01 00:04:00,180.0,37.639167,-120.995833


In [40]:
flights.head()

,FL_DATE,DEP_TIME,ARR_TIME,o_lat,o_lng,d_lat,d_lng
0,2013-01-01,71820,82680,40.690292,-73.988544,26.640628,-81.872308
1,2013-01-01,43140,53940,40.690292,-73.988544,26.640628,-81.872308
2,2013-01-01,56640,67140,40.690292,-73.988544,26.640628,-81.872308
3,2013-01-01,59580,68820,26.640628,-81.872308,40.690292,-73.988544
4,2013-01-01,70140,79560,26.640628,-81.872308,40.690292,-73.988544
